In [1]:
import os
import json
import logging
from glob import glob
import pandas as pd
from infrastructure.storage.swift import Swift
from config.harvester_config import config_harvester

# create logger
logger = logging.getLogger()
logger.setLevel(logging.WARNING)
PROCESSED_PUBLICATIONS_DIR = './processed_publications/'

In [2]:
def load_softcite_output_data(processed_files, batch_size=100):
    clean_up()
    partitions = [file for i in range(0,len(processed_files),batch_size) for file in processed_files[i:i+batch_size] if file.endswith('.software.json')]
    mentions = []
    for partition in partitions:
        local_files = download_batch(partition, PROCESSED_PUBLICATIONS_DIR)
        files_content = load_content(local_files)
        mentions.extend(extract_mentions(files_content))
        clean_up()
    return mentions


def clean_up():
    for file in glob(os.path.join(PROCESSED_PUBLICATIONS_DIR, '*.software.json')):
        os.remove(file)

def extract_mentions(files_content):
    mentions = []
    for file_id, json_file in files_content:
        file_mentions = []
        for mention in json_file['mentions']:
            file_mentions.append(mention['software-name']['normalizedForm'])
        mentions.append((file_id, file_mentions))
    return mentions
    
def load_content(local_files):
    files_content = []
    for file in local_files:
        file_id = os.path.basename(file).split('.')[0]
        with open(file, 'r') as f:
            files_content.append((file_id, json.load(f)))
    return files_content

def download_batch(batch, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    storage_handler.download_files(config_harvester['publications_dump'], batch, dest_path=dest_dir)
    return glob(os.path.join(PROCESSED_PUBLICATIONS_DIR, '*.software.json'))

In [3]:
storage_handler = Swift(config_harvester)

using output SWIFT bso3_publications_dump container: {'count': 26573, 'last_modified': '2022-02-08T15:52:53.709830', 'bytes': 19343955200, 'name': 'bso3_publications_dump'}
using input SWIFT bso_dump container: {'count': 117, 'last_modified': '2022-02-05T17:36:08.026270', 'bytes': 4833008355, 'name': 'bso_dump'}
2022-02-08 17:02:56,329 | infrastructure.storage.swift | DEBUG | container already exists on SWIFT object storage: bso3_publications_dump


In [4]:
processed_files = storage_handler.get_swift_list(config_harvester['publications_dump'], dir_name='processed')
print(f"Il y a {len(processed_files)} fichiers")

Il y a 7297 fichiers


In [5]:
# 7297 fichiers => ~15min
%time mentions = load_softcite_output_data(processed_files)
# %time mentions = load_softcite_output_data(processed_files[:200])

2022-02-08 17:02:59,332 | infrastructure.storage.swift | DEBUG | downloading processed/00/05/4b/55/00054b55-3bf8-45a5-9c44-9bdbaf4d60df.software/00054b55-3bf8-45a5-9c44-9bdbaf4d60df.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:00,560 | infrastructure.storage.swift | DEBUG | downloading processed/00/08/49/c1/000849c1-e55e-48e2-ab83-6dfdbd323d5b.software/000849c1-e55e-48e2-ab83-6dfdbd323d5b.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:00,658 | infrastructure.storage.swift | DEBUG | downloading processed/00/0f/e7/b5/000fe7b5-8930-45e4-8d55-bfbdb37c4165.software/000fe7b5-8930-45e4-8d55-bfbdb37c4165.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:00,767 | infrastructure.storage.swift | DEBUG | downloading processed/00/16/1c/7a/00161c7a-ac7f-47c4-af6c-9bb81e9ec80e.software/00161c7a-ac7f-47c4-af6c-9bb81e9ec80e.software.json from cont

2022-02-08 17:03:04,594 | infrastructure.storage.swift | DEBUG | downloading processed/00/ba/58/ed/00ba58ed-d2d3-4686-a715-4fa3016b67b7.software/00ba58ed-d2d3-4686-a715-4fa3016b67b7.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:04,660 | infrastructure.storage.swift | DEBUG | downloading processed/00/ba/eb/6a/00baeb6a-4f51-453e-95dc-b14f2f759ed2.software/00baeb6a-4f51-453e-95dc-b14f2f759ed2.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:04,769 | infrastructure.storage.swift | DEBUG | downloading processed/00/cb/4b/9a/00cb4b9a-caba-4a7d-b163-0bd02937ebeb.software/00cb4b9a-caba-4a7d-b163-0bd02937ebeb.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:05,050 | infrastructure.storage.swift | DEBUG | downloading processed/00/cb/7f/74/00cb7f74-db96-4d8d-807a-48b354ff62d8.software/00cb7f74-db96-4d8d-807a-48b354ff62d8.software.json from cont

2022-02-08 17:03:07,970 | infrastructure.storage.swift | DEBUG | downloading processed/01/64/13/40/01641340-f759-417c-8ba5-303bd48fcb1a.software/01641340-f759-417c-8ba5-303bd48fcb1a.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:08,081 | infrastructure.storage.swift | DEBUG | downloading processed/01/6c/32/9d/016c329d-51e8-43f3-88f9-00c7dbe0dfd3.software/016c329d-51e8-43f3-88f9-00c7dbe0dfd3.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:08,161 | infrastructure.storage.swift | DEBUG | downloading processed/01/6f/65/9f/016f659f-038e-4aea-b98c-53db44235327.software/016f659f-038e-4aea-b98c-53db44235327.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:08,304 | infrastructure.storage.swift | DEBUG | downloading processed/01/71/fe/90/0171fe90-982e-4d96-b245-1c97cfbfc484.software/0171fe90-982e-4d96-b245-1c97cfbfc484.software.json from cont

2022-02-08 17:03:10,593 | infrastructure.storage.swift | DEBUG | downloading processed/02/7c/10/27/027c1027-fce3-4aa1-91d5-fd917f7a8fb6.software/027c1027-fce3-4aa1-91d5-fd917f7a8fb6.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:10,662 | infrastructure.storage.swift | DEBUG | downloading processed/02/7c/9f/ad/027c9fad-507e-43b6-a090-0a11a4fff0cd.software/027c9fad-507e-43b6-a090-0a11a4fff0cd.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:10,754 | infrastructure.storage.swift | DEBUG | downloading processed/02/80/5f/57/02805f57-7a3e-453f-8fe2-41cae63e7991.software/02805f57-7a3e-453f-8fe2-41cae63e7991.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:10,830 | infrastructure.storage.swift | DEBUG | downloading processed/02/81/49/75/02814975-39bd-486b-bfe2-bd0e1286c85b.software/02814975-39bd-486b-bfe2-bd0e1286c85b.software.json from cont

2022-02-08 17:03:13,723 | infrastructure.storage.swift | DEBUG | downloading processed/03/4e/ea/3b/034eea3b-44de-4398-a734-4acba5b8eab4.software/034eea3b-44de-4398-a734-4acba5b8eab4.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:13,828 | infrastructure.storage.swift | DEBUG | downloading processed/03/57/78/d3/035778d3-be28-4fb1-b425-a0cc213d927f.software/035778d3-be28-4fb1-b425-a0cc213d927f.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:13,895 | infrastructure.storage.swift | DEBUG | downloading processed/03/5b/8b/4b/035b8b4b-414b-48e1-a522-900971fcbd35.software/035b8b4b-414b-48e1-a522-900971fcbd35.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:03:13,950 | infrastructure.storage.swift | DEBUG | downloading processed/03/6c/c6/86/036cc686-da91-4b43-8ca4-11827f4c5771.software/036cc686-da91-4b43-8ca4-11827f4c5771.software.json from cont

...

2022-02-08 17:18:13,229 | infrastructure.storage.swift | DEBUG | downloading processed/bd/6e/ab/1e/bd6eab1e-49cb-4016-8cda-430be3edb02f.software/bd6eab1e-49cb-4016-8cda-430be3edb02f.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:13,318 | infrastructure.storage.swift | DEBUG | downloading processed/bd/73/e9/8a/bd73e98a-d26d-42fb-8b8f-873aa7f3b3bf.software/bd73e98a-d26d-42fb-8b8f-873aa7f3b3bf.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:13,382 | infrastructure.storage.swift | DEBUG | downloading processed/bd/77/82/40/bd778240-24fe-498f-975b-738ef32933e9.software/bd778240-24fe-498f-975b-738ef32933e9.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:13,508 | infrastructure.storage.swift | DEBUG | downloading processed/bd/82/67/75/bd826775-6eb9-4046-8f26-88a02d903efd.software/bd826775-6eb9-4046-8f26-88a02d903efd.software.json from cont

2022-02-08 17:18:17,097 | infrastructure.storage.swift | DEBUG | downloading processed/be/60/33/5f/be60335f-b823-480f-ad99-86809664ab24.software/be60335f-b823-480f-ad99-86809664ab24.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:17,228 | infrastructure.storage.swift | DEBUG | downloading processed/be/68/5a/44/be685a44-3b07-4a79-afbc-8a48b59d8e66.software/be685a44-3b07-4a79-afbc-8a48b59d8e66.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:17,400 | infrastructure.storage.swift | DEBUG | downloading processed/be/6b/bd/83/be6bbd83-a48d-4c40-8208-023538974c6d.software/be6bbd83-a48d-4c40-8208-023538974c6d.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:17,522 | infrastructure.storage.swift | DEBUG | downloading processed/be/72/c3/8e/be72c38e-7a95-40cb-a311-b1c0a7808c57.software/be72c38e-7a95-40cb-a311-b1c0a7808c57.software.json from cont

2022-02-08 17:18:21,078 | infrastructure.storage.swift | DEBUG | downloading processed/bf/11/fb/91/bf11fb91-cf61-4196-88dd-891f29588839.software/bf11fb91-cf61-4196-88dd-891f29588839.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:21,161 | infrastructure.storage.swift | DEBUG | downloading processed/bf/13/ee/08/bf13ee08-c4b6-4f48-a60d-aeda54a297ce.software/bf13ee08-c4b6-4f48-a60d-aeda54a297ce.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:21,245 | infrastructure.storage.swift | DEBUG | downloading processed/bf/1b/2c/85/bf1b2c85-0eac-4327-984d-b734520382f2.software/bf1b2c85-0eac-4327-984d-b734520382f2.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:21,322 | infrastructure.storage.swift | DEBUG | downloading processed/bf/1d/92/2e/bf1d922e-569a-4a38-84c1-e86d841d3252.software/bf1d922e-569a-4a38-84c1-e86d841d3252.software.json from cont

2022-02-08 17:18:26,034 | infrastructure.storage.swift | DEBUG | downloading processed/bf/fd/07/b7/bffd07b7-9b92-43ca-8dc1-6f241d780543.software/bffd07b7-9b92-43ca-8dc1-6f241d780543.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:26,718 | infrastructure.storage.swift | DEBUG | downloading processed/c0/01/23/f6/c00123f6-a722-4668-9c6e-a0d855d219bc.software/c00123f6-a722-4668-9c6e-a0d855d219bc.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:26,834 | infrastructure.storage.swift | DEBUG | downloading processed/c0/08/8e/c2/c0088ec2-baeb-4d20-b6ca-edfcd1d9bada.software/c0088ec2-baeb-4d20-b6ca-edfcd1d9bada.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:27,027 | infrastructure.storage.swift | DEBUG | downloading processed/c0/32/b8/3d/c032b83d-cf95-4337-834f-39379925824f.software/c032b83d-cf95-4337-834f-39379925824f.software.json from cont

2022-02-08 17:18:30,045 | infrastructure.storage.swift | DEBUG | downloading processed/c0/e7/34/cd/c0e734cd-bced-4140-bbc7-a9d77fde5e6b.software/c0e734cd-bced-4140-bbc7-a9d77fde5e6b.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:30,145 | infrastructure.storage.swift | DEBUG | downloading processed/c0/e8/67/8b/c0e8678b-066b-44c7-8cd6-cd706d7cb684.software/c0e8678b-066b-44c7-8cd6-cd706d7cb684.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:30,237 | infrastructure.storage.swift | DEBUG | downloading processed/c0/ed/c8/62/c0edc862-d82d-4294-8000-5cfa3fa5c658.software/c0edc862-d82d-4294-8000-5cfa3fa5c658.software.json from container bso3_publications_dump into ./processed_publications/
2022-02-08 17:18:30,313 | infrastructure.storage.swift | DEBUG | downloading processed/c1/0d/dd/44/c10ddd44-8f3f-4473-9574-f13e6baffd59.software/c10ddd44-8f3f-4473-9574-f13e6baffd59.software.json from cont

In [6]:
mentions_by_files = pd.DataFrame(mentions).rename(columns={0:'file_id', 1:'mentions'}).set_index('file_id')
mentions_by_files['nb_mentions'] = mentions_by_files['mentions'].apply(len)
mentions_by_files['nb_unique_mentions'] = mentions_by_files['mentions'].apply(set).apply(len)
mentions_by_files.describe()

,nb_mentions,nb_unique_mentions
count,7293.000000,7293.000000
mean,4.147127,1.898396
std,15.118544,3.374254
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,1.000000
75%,3.000000,2.000000
max,502.000000,37.000000


In [10]:
mentions_by_files.head()

,mentions,nb_mentions,nb_unique_mentions
file_id,,,
00054b55-3bf8-45a5-9c44-9bdbaf4d60df,"[Cutadapt, fastQC_0, GraphPad Prism, DESeq]",4,4
000849c1-e55e-48e2-ab83-6dfdbd323d5b,"[SPSSv, MedCalc]",2,2
000fe7b5-8930-45e4-8d55-bfbdb37c4165,"[Volocity, DIAS, ImageJ, MATLAB, Photo- shop, ...",12,7
00161c7a-ac7f-47c4-af6c-9bb81e9ec80e,[],0,0
001f0168-f4a3-4ff4-a98a-cf38216e9aec,[],0,0


In [7]:
mentions_count = pd.Series([e for file_id, mentions_list in mentions for e in mentions_list]).value_counts()
print(mentions_count.describe())
mentions_count

count    7709.000000
mean        3.923336
std        13.142169
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max       479.000000
dtype: float64


ImageJ                      479
SAS                         297
SPEDAS                      285
MATLAB                      255
GraphPad Prism              254
                           ... 
MS Office Excel               1
snpEff                        1
vcf-annotate-filter Qual      1
Viewer                        1
GESTE                         1
Length: 7709, dtype: int64

In [8]:
u_mentions_count = pd.Series([e for file_id, mentions_list in mentions for e in set(mentions_list)]).value_counts()
print(u_mentions_count.describe())
u_mentions_count

count    7709.000000
mean        1.795953
std         6.806065
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max       288.000000
dtype: float64


ImageJ                     288
SAS                        235
GraphPad Prism             219
SPSS                       210
Matlab                     139
                          ... 
Surveillance R-package       1
Surveillance R- package      1
NIS-elements AR              1
xspec                        1
GESTE                        1
Length: 7709, dtype: int64